## AgentiC RAG for Math education with human feedback

In [ ]:
# Environment Setup and Dependencies
%pip install -q langchain-community langchain-core langchain-text-splitters
%pip install -q qdrant-client sentence-transformers
%pip install -q dspy-ai langchain-tavily portkey-ai
%pip install -q groq datasets pandas numpy
%pip install -q langchain-qdrant langgraph
%pip install -q langchain-openai langchain-huggingface
%pip install -q langgraph[checkpoint]

In [ ]:
import os
import json
import pandas as pd
import numpy as np
from typing import List, Dict, Any, Optional, TypedDict, Annotated
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Environment variables setup
os.environ["GROQ_API_KEY"] = "your groq api key"
os.environ["TAVILY_API_KEY"] = "tavily api key"
os.environ["PORTKEY_API_KEY"] = "portkey api key"

## Data Loading and Preprocessing using JEE Bench dataset

In [ ]:
from datasets import load_dataset

In [ ]:
def load_jee_bench_data():
    """Load and preprocess JEE Bench dataset, keeping only math questions"""
    try:
        dataset = load_dataset("daman1209arora/jeebench")
        df = dataset['test'].to_pandas()
        
        df['subject'] = df['subject'].str.lower()
        math_keywords = ['math', 'mathematics', 'algebra', 'calculus', 'geometry', 'trigonometry']
        math_df = df[df['subject'].str.contains('|'.join(math_keywords), case=False, na=False)].copy()
        
        print(f"Found {len(math_df)} math questions out of {len(df)} total questions")
        
        knowledge_base = []
        for idx, row in math_df.iterrows():
            entry = {
                'id': f"jee_math_{idx}",
                'question': row.get('question', 'Question not available'),
                'description': row.get('description', ''),
                'answer': row.get('gold', ''),
                'topic': row.get('type', 'General'),
                'difficulty': 'Medium',
                'metadata': {
                    'source': 'JEE_Bench',
                    'subject': row.get('subject', 'Mathematics'),
                    'index': row.get('index', idx)
                }
            }
            knowledge_base.append(entry)
        
        print(f"Successfully loaded {len(knowledge_base)} math questions into knowledge base")
        return knowledge_base
    
    except Exception as e:
        print(f"Error loading JEE Bench data: {str(e)}")
        return []


In [ ]:
# Load the knowledge base
knowledge_base = load_jee_bench_data()

## Portkey Guardrails Setup

In [ ]:
from langchain_openai import ChatOpenAI
from portkey_ai import createHeaders

In [ ]:
# Define guardrails configuration
# INPUT GUARDRAILS CONFIGURATION
input_guardrail_config = {
    "before_request_hooks": [
        {
            "type": "guardrail",
            "id": "math-input-guardrail",
            "checks": [
                {
                    "id": "default.regexMatch",
                    "parameters": {
                        "pattern": r".*(equation|solve|derivative|integral|limit|matrix|probability|geometry|algebra|calculus|trigonometry|statistics|graph|function|find|calculate|determine|evaluate|area|perimeter|volume|radius|diameter|triangle|circle|rectangle|square|angle|pythagorean|theorem|sin|cos|tan|mathematics|math|formula|explain|prove|show|demonstrate).*",
                        "match_type": "contains",
                        "case_sensitive": False
                    },
                    "on_fail": "deny"
                },
                {
                    "id": "default.detectGibberish",
                    "on_fail": "deny"
                },
                {
                    "id": "default.characterCount",
                    "parameters": {
                        "min_length": 10,
                        "max_length": 1000
                    },
                    "on_fail": "deny"
                }
            ]
        }
    ]
}


In [ ]:
# OUTPUT GUARDRAILS CONFIGURATION
output_guardrail_config = {
    "after_request_hooks": [
        {
            "type": "guardrail", 
            "id": "math-output-validation",
            "checks": [
                {
                    "id": "default.contains",
                    "parameters": {
                        "required_phrases": ["Step", "Solution", "Therefore", "Answer"],
                        "match_type": "any"  # At least one phrase must be present
                    },
                    "on_fail": "retry"
                },
                {
                    "id": "default.characterCount",
                    "parameters": {
                        "min_length": 100,  # Ensure detailed solutions
                        "max_length": 3000
                    },
                    "on_fail": "log"
                },
                {
                    "id": "default.regexMatch",
                    "parameters": {
                        "pattern": r".*(step|solve|answer|therefore|hence|thus|final|result|solution).*",
                        "match_type": "contains",
                        "case_sensitive": False
                    },
                    "on_fail": "retry"
                },
                {
                    "id": "default.detectGibberish",
                    "on_fail": "retry"
                }
            ]
        }
    ]
}


In [ ]:
# Setup separate LLMs for input and output guardrails
llm_input_guardrails = ChatOpenAI(
    api_key=os.environ.get("GROQ_API_KEY"),
    base_url="https://api.portkey.ai/v1",
    default_headers=createHeaders(
        api_key=os.environ.get("PORTKEY_API_KEY"),
        provider="groq",
        config=input_guardrail_config
    ),
    model="moonshotai/kimi-k2-instruct",
    temperature=0.1,
    model_kwargs={"max_tokens": 100}  # Short responses for validation
)

In [ ]:
llm_output_guardrails = ChatOpenAI(
    api_key=os.environ.get("GROQ_API_KEY"),
    base_url="https://api.portkey.ai/v1", 
    default_headers=createHeaders(
        api_key=os.environ.get("PORTKEY_API_KEY"),
        provider="groq",
        config=output_guardrail_config
    ),
    model="moonshotai/kimi-k2-instruct",
    temperature=0.1,
    model_kwargs={"max_tokens": 2000}  # Longer responses for solutions
)

## Vector Database setup with qdrant

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

In [ ]:
# Initialize embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Initialize Qdrant client (in-memory)
client = QdrantClient(":memory:")

In [ ]:
# Create collection
client.create_collection(
    collection_name="jee_math_problems",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

In [ ]:
# Create vector store
vector_store = QdrantVectorStore(
    client=client,
    collection_name="jee_math_problems",
    embedding=embeddings,
)

In [ ]:
# Convert knowledge base to documents
documents = []
metadatas = []
for item in knowledge_base:
    content = f"Question: {item['question']}"
    if item.get('description'):
        content += f"\nDescription: {item['description']}"
    if item.get('answer'):
        content += f"\nAnswer: {item['answer']}"
    
    documents.append(content)
    metadata = {
        'id': item['id'],
        'question': item['question'],
        'description': item.get('description', ''),
        'answer': item['answer'],
        'topic': item['topic'],
        'difficulty': item['difficulty'],
        'source': item['metadata']['source'],
        'subject': item['metadata']['subject'],
        'index': item['metadata']['index']
    }
    metadatas.append(metadata)

In [ ]:
# Add documents to vector store
print(f"Adding {len(documents)} documents to Qdrant vector store...")
vector_store.add_texts(texts=documents, metadatas=metadatas)
print("Documents added successfully!")

## Web Search setup with tavily

In [ ]:
from langchain_tavily import TavilySearch

In [ ]:
tavily_search = TavilySearch(
    max_results=5,
    topic="general",
    include_answer=True,
    search_depth="advanced",
    include_domains=["mathway.com", "wolframalpha.com", "khanacademy.org", "symbolab.com"]
)


## DSPy setup for Human feedback

In [ ]:
import dspy

In [ ]:
# Configure DSPy with Groq
lm = dspy.LM(model="groq/moonshotai/kimi-k2-instruct", api_key=os.environ.get("GROQ_API_KEY"))
dspy.configure(lm=lm)

In [ ]:
class DSPyMathOptimizer:
    """DSPy-based optimizer for math education with human feedback"""
    
    def __init__(self):
        self.rag_module = self._create_rag_module()
        self.feedback_data = []
        
    def _create_rag_module(self):
        """Create DSPy RAG module for math problems"""
        
        class MathRAG(dspy.Module):
            def __init__(self):
                super().__init__()
                self.generate_answer = dspy.ChainOfThought("question, context -> solution")
            
            def forward(self, question, context=""):
                prediction = self.generate_answer(question=question, context=context)
                return dspy.Prediction(solution=prediction.solution)
        
        return MathRAG()
    
    def solve_problem(self, question: str, context: str = ""):
        """Solve a math problem using the RAG module"""
        try:
            result = self.rag_module(question=question, context=context)
            return result.solution
        except Exception as e:
            return f"Error solving problem: {e}"
    
    def collect_feedback(self, question: str, generated_solution: str, rating: int, comments: dict):
        """Collect human feedback"""
        feedback_entry = {
            "question": question,
            "generated_solution": generated_solution,
            "rating": rating,
            "comments": comments,
            "timestamp": pd.Timestamp.now()
        }
        self.feedback_data.append(feedback_entry)
        print(f" Feedback collected: Rating {rating}/5")

In [ ]:
# Initialize DSPy optimizer
dspy_optimizer = DSPyMathOptimizer()

## Langgraph workflow implementation

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, SystemMessage

In [ ]:
class AgenticRAGState(TypedDict):
    """State for the Agentic RAG workflow"""
    user_question: str
    input_guardrails_passed: bool
    output_guardrails_passed: bool
    knowledge_base_results: List[Dict]
    web_search_results: List[Dict]
    raw_solution: str
    final_solution: str
    feedback_rating: Optional[int]
    feedback_comments: Optional[Dict]
    error_message: Optional[str]
    guardrail_attempts: int

In [ ]:
def input_guardrails_node(state: AgenticRAGState) -> AgenticRAGState:
    """Node 1: Apply INPUT guardrails using Portkey - FIXED VERSION"""
    try:
        print("Applying INPUT guardrails...")
        
        # math keywords - comprehensive list
        math_keywords = [
            # Basic operations
            'equation', 'solve', 'find', 'calculate', 'determine', 'evaluate', 'compute',
            # Algebra
            'algebra', 'polynomial', 'quadratic', 'linear', 'variable', 'coefficient',
            # Calculus
            'derivative', 'integral', 'limit', 'differential', 'antiderivative',
            # Geometry
            'geometry', 'area', 'perimeter', 'volume', 'radius', 'diameter', 'triangle', 
            'circle', 'rectangle', 'square', 'angle', 'pythagorean', 'theorem',
            # Trigonometry
            'trigonometry', 'sin', 'cos', 'tan', 'sine', 'cosine', 'tangent',
            # General math
            'mathematics', 'math', 'formula', 'function', 'graph', 'plot',
            # Statistics
            'statistics', 'probability', 'mean', 'median', 'mode', 'deviation',
            # Other math concepts
            'matrix', 'vector', 'logarithm', 'exponential', 'factorial', 'prime'
        ]
        
        # Math operation verbs
        math_verbs = ['explain', 'prove', 'show', 'demonstrate', 'derive', 'verify']
        
        question_lower = state["user_question"].lower()
        
        # Check for math keywords
        has_math_keyword = any(keyword in question_lower for keyword in math_keywords)
        
        # Check for math operation verbs with math context
        has_math_verb = any(verb in question_lower for verb in math_verbs)
        
        # Check for mathematical expressions (numbers, symbols)
        import re
        has_math_symbols = bool(re.search(r'[0-9+\-*/=^()x²³√∫∂∑π]', question_lower))
        
        # More lenient validation - pass if ANY condition is met
        is_math_question = has_math_keyword or (has_math_verb and has_math_symbols) or has_math_symbols
        
        # Additional Portkey validation
        try:
            test_message = [
                SystemMessage(content="Respond 'VALID' if this is a math question, 'INVALID' otherwise."),
                HumanMessage(content=state["user_question"])
            ]
            response = llm_input_guardrails.invoke(test_message)
            portkey_validation = "VALID" in response.content.upper()
        except:
            portkey_validation = True  # Default to true if Portkey fails
        
        # Final decision - pass if either validation method succeeds
        input_passed = is_math_question or portkey_validation
        
        print(f"   Math keywords: {has_math_keyword}")
        print(f"   Math symbols: {has_math_symbols}")
        print(f"   Portkey validation: {portkey_validation}")
        print(f"   Input guardrails: {'PASSED' if input_passed else 'FAILED'}")
        
        return {
            **state,
            "input_guardrails_passed": input_passed,
            "error_message": None if input_passed else "Question failed input validation - not a valid math question"
        }
    except Exception as e:
        print(f"Input guardrails error: {str(e)}")
        return {
            **state,
            "input_guardrails_passed": False,
            "error_message": f"Input guardrails error: {str(e)}"
        }



In [ ]:
def vector_search_node(state: AgenticRAGState) -> AgenticRAGState:
    """Node 2: Search in knowledge base using vector search"""
    if not state["input_guardrails_passed"]:
        return state
    
    try:
        print("Searching knowledge base...")
        
        # Search for similar problems in knowledge base
        results = vector_store.similarity_search_with_score(state["user_question"], k=3)
        
        knowledge_results = []
        for doc, score in results:
            result = {
                'question': doc.metadata['question'],
                'answer': doc.metadata['answer'],
                'topic': doc.metadata['topic'],
                'score': float(score),
                'content': doc.page_content
            }
            knowledge_results.append(result)
        
        print(f"   Found {len(knowledge_results)} similar problems")
        
        return {
            **state,
            "knowledge_base_results": knowledge_results
        }
    except Exception as e:
        print(f"Vector search error: {str(e)}")
        return {
            **state,
            "error_message": f"Vector search error: {str(e)}"
        }


In [ ]:
def web_search_node(state: AgenticRAGState) -> AgenticRAGState:
    """Node 3: Perform web search using Tavily"""
    if not state["input_guardrails_passed"]:
        return state
    
    try:
        print("Performing web search...")
        
        # Only do web search if no good knowledge base results found
        should_search = (not state["knowledge_base_results"] or 
                        state["knowledge_base_results"][0]["score"] > 0.7)
        
        if should_search:
            search_query = f"solve step by step math problem: {state['user_question']}"
            web_results = tavily_search.invoke({"query": search_query})
            
            processed_results = []
            for result in web_results.get('results', [])[:3]:  # Take top 3 results
                processed_results.append({
                    'title': result.get('title', ''),
                    'content': result.get('content', '')[:500],  # Limit content length
                    'url': result.get('url', '')
                })
            
            print(f"   Found {len(processed_results)} web resources")
            
            return {
                **state,
                "web_search_results": processed_results
            }
        else:
            print("   Skipped web search - good knowledge base match found")
            return {
                **state,
                "web_search_results": []
            }
    except Exception as e:
        print(f"Web search error: {str(e)}")
        return {
            **state,
            "error_message": f"Web search error: {str(e)}"
        }

In [ ]:
def solution_generation_node(state: AgenticRAGState) -> AgenticRAGState:
    """Node 4: Generate raw solution using DSPy (before output guardrails)"""
    if not state["input_guardrails_passed"]:
        return state
    
    try:
        print("Generating initial solution...")
        
        # Prepare context from knowledge base and web results
        context = ""
        
        if state["knowledge_base_results"]:
            context += "Knowledge Base Results:\n"
            for i, result in enumerate(state["knowledge_base_results"][:2], 1):
                context += f"{i}. Question: {result['question']}\n   Answer: {result['answer']}\n\n"
        
        if state["web_search_results"]:
            context += "Web Search Results:\n"
            for i, result in enumerate(state["web_search_results"][:2], 1):
                context += f"{i}. {result['title']}\n   Content: {result['content']}\n\n"
        
        # Generate raw solution using DSPy
        if context:
            raw_solution = dspy_optimizer.solve_problem(state["user_question"], context)
        else:
            raw_solution = dspy_optimizer.solve_problem(state["user_question"])
        
        print(f"   Generated solution: {len(raw_solution)} characters")
        
        return {
            **state,
            "raw_solution": raw_solution,
            "guardrail_attempts": 0
        }
        
    except Exception as e:
        print(f"Solution generation error: {str(e)}")
        return {
            **state,
            "error_message": f"Solution generation error: {str(e)}"
        }

In [ ]:
def output_guardrails_node(state: AgenticRAGState) -> AgenticRAGState:
    """Node 5: Apply OUTPUT guardrails using Portkey"""
    if not state["input_guardrails_passed"] or not state["raw_solution"]:
        return state
    
    try:
        print("Applying OUTPUT guardrails...")
        
        # Create a proper solution prompt for guardrails
        solution_prompt = f"""
Please format this math solution properly with clear steps:

Original Question: {state["user_question"]}

Raw Solution: {state["raw_solution"]}

Requirements:
1. Start with "Solution:"
2. Break down into numbered steps
3. Show all calculations clearly
4. End with "Therefore, the final answer is..."
5. Use proper mathematical notation

Please provide a well-structured solution:
"""
        
        max_attempts = 3
        attempt = state.get("guardrail_attempts", 0)
        
        while attempt < max_attempts:
            try:
                print(f"   Attempt {attempt + 1}/{max_attempts}")
                
                # This will trigger output guardrails
                guardrailed_response = llm_output_guardrails.invoke([
                    SystemMessage(content="You are a math tutor providing step-by-step solutions. Always follow the formatting requirements."),
                    HumanMessage(content=solution_prompt)
                ])
                
                final_solution = guardrailed_response.content
                
                # Additional manual validation
                required_elements = ["step", "solution", "answer"]
                has_elements = any(element.lower() in final_solution.lower() for element in required_elements)
                is_long_enough = len(final_solution) >= 100
                
                if has_elements and is_long_enough:
                    print("Output guardrails PASSED")
                    return {
                        **state,
                        "final_solution": final_solution,
                        "output_guardrails_passed": True,
                        "guardrail_attempts": attempt + 1
                    }
                else:
                    attempt += 1
                    print(f"Output validation failed on attempt {attempt}")
                    
            except Exception as guardrail_error:
                attempt += 1
                print(f"Output guardrails triggered on attempt {attempt}: {str(guardrail_error)}")
        
        # Fallback: Use manual formatting if all attempts fail
        print("Using fallback manual formatting")
        fallback_solution = format_solution_manually(state["raw_solution"], state["user_question"])
        
        return {
            **state,
            "final_solution": fallback_solution,
            "output_guardrails_passed": False,
            "guardrail_attempts": max_attempts,
            "error_message": "Output guardrails failed, used fallback formatting"
        }
        
    except Exception as e:
        print(f"Output guardrails error: {str(e)}")
        fallback_solution = format_solution_manually(
            state.get("raw_solution", "No solution generated"), 
            state["user_question"]
        )
        return {
            **state,
            "final_solution": fallback_solution,
            "output_guardrails_passed": False,
            "error_message": f"Output guardrails error: {str(e)}"
        }

def format_solution_manually(solution: str, question: str) -> str:
    """Manual formatting when output guardrails fail"""
    formatted_solution = f"""Solution:

Step 1: Understanding the Problem
{question}

Step 2: Detailed Solution
{solution}

Step 3: Verification and Final Answer
The solution above provides the mathematical approach and reasoning needed to solve this problem.

Therefore, the final answer is contained in the detailed solution provided in Step 2.

Note: This solution has been manually formatted to meet quality standards after guardrails processing.
"""
    return formatted_solution


In [ ]:
def feedback_collection_node(state: AgenticRAGState) -> AgenticRAGState:
    """Node 6: Collect human feedback (simulated for demo)"""
    if not state["input_guardrails_passed"] or not state["final_solution"]:
        return state
    
    try:
        print("Collecting feedback...")
        
        # In a real application, this would collect actual user feedback
        # For demo purposes, we'll simulate feedback based on output guardrails performance
        if state.get("output_guardrails_passed", False):
            simulated_feedback = {
                "rating": 5,
                "comments": {
                    "clarity": "Excellent step-by-step explanation",
                    "accuracy": "Mathematically correct and well-formatted",
                    "completeness": "Complete solution with proper structure"
                }
            }
        else:
            simulated_feedback = {
                "rating": 3,
                "comments": {
                    "clarity": "Solution provided but formatting could be improved",
                    "accuracy": "Mathematically correct",
                    "completeness": "Used fallback formatting due to guardrails"
                }
            }
        
        # Collect feedback using DSPy optimizer
        dspy_optimizer.collect_feedback(
            state["user_question"],
            state["final_solution"],
            simulated_feedback["rating"],
            simulated_feedback["comments"]
        )
        
        print(f"   Rating: {simulated_feedback['rating']}/5")
        
        return {
            **state,
            "feedback_rating": simulated_feedback["rating"],
            "feedback_comments": simulated_feedback["comments"]
        }
    except Exception as e:
        print(f"Feedback collection error: {str(e)}")
        return state

In [ ]:
# Build the LangGraph workflow
workflow = StateGraph(AgenticRAGState)

In [ ]:
# Add nodes
workflow.add_node("input_guardrails", input_guardrails_node)
workflow.add_node("vector_search", vector_search_node)
workflow.add_node("web_search", web_search_node)
workflow.add_node("solution_generation", solution_generation_node)
workflow.add_node("output_guardrails", output_guardrails_node)  
workflow.add_node("feedback_collection", feedback_collection_node)


In [ ]:
# Add edges
workflow.add_edge(START, "input_guardrails")
workflow.add_edge("input_guardrails", "vector_search")
workflow.add_edge("vector_search", "web_search")
workflow.add_edge("web_search", "solution_generation")
workflow.add_edge("solution_generation", "output_guardrails")
workflow.add_edge("output_guardrails", "feedback_collection")  
workflow.add_edge("feedback_collection", END)

In [ ]:
# Compile the workflow
app = workflow.compile()

In [ ]:
from IPython.display import Image, display
display(Image(app.get_graph(xray=True).draw_mermaid_png()))


## Complete Agentic RAG System

In [ ]:
class AgenticMathRAG:
    """Complete Agentic RAG system for Math Education with Proper Output Guardrails"""
    
    def __init__(self):
        self.workflow_app = app
        self.dspy_optimizer = dspy_optimizer
    
    def solve_math_problem(self, question: str) -> Dict[str, Any]:
        """Main method to solve math problems using the complete workflow"""
        
        # Initialize state
        initial_state = AgenticRAGState(
            user_question=question,
            input_guardrails_passed=False,
            output_guardrails_passed=False,
            knowledge_base_results=[],
            web_search_results=[],
            raw_solution="",
            final_solution="",
            feedback_rating=None,
            feedback_comments=None,
            error_message=None,
            guardrail_attempts=0
        )
        
        print(f"Processing question: {question}")
        print("=" * 80)
        
        # Run the workflow
        try:
            final_state = self.workflow_app.invoke(initial_state)
            
            # Display results
            self._display_results(final_state)
            
            return final_state
        
        except Exception as e:
            print(f"Workflow error: {str(e)}")
            return {"error": str(e)}
    
    def _display_results(self, state: AgenticRAGState):
        """Display the results of the workflow"""
        
        print("\n**INPUT GUARDRAILS:**")
        if state["input_guardrails_passed"]:
            print("Input passed guardrails - Valid math question")
        else:
            print(f"Input failed guardrails - {state.get('error_message', 'Invalid input')}")
            return
        
        print("\n**KNOWLEDGE BASE SEARCH:**")
        if state["knowledge_base_results"]:
            print(f"Found {len(state['knowledge_base_results'])} similar problems")
            for i, result in enumerate(state["knowledge_base_results"][:2], 1):
                print(f"{i}. Score: {result['score']:.3f} - Topic: {result['topic']}")
        else:
            print("No similar problems found in knowledge base")
        
        print("\n**WEB SEARCH:**")
        if state["web_search_results"]:
            print(f"Found {len(state['web_search_results'])} web resources")
            for i, result in enumerate(state["web_search_results"][:2], 1):
                print(f"{i}. {result['title'][:50]}...")
        else:
            print("No web search performed (good knowledge base match found)")
        
        print("\n**OUTPUT GUARDRAILS:**")
        if state.get("output_guardrails_passed", False):
            print(f"Output passed guardrails after {state.get('guardrail_attempts', 0)} attempts")
        else:
            print(f"Output guardrails failed after {state.get('guardrail_attempts', 0)} attempts - used fallback")
        
        print("\n**FINAL SOLUTION:**")
        solution_preview = state['final_solution'][:300] + "..." if len(state['final_solution']) > 300 else state['final_solution']
        print(f"{solution_preview}")
        
        print("\n**FEEDBACK:**")
        if state["feedback_rating"]:
            print(f"Rating: {state['feedback_rating']}/5")
            if state["feedback_comments"]:
                for aspect, comment in state["feedback_comments"].items():
                    print(f"• {aspect.capitalize()}: {comment}")
        
        print("\n" + "=" * 80)

In [ ]:
# Initialize the complete system
math_rag_system = AgenticMathRAG()

## Testing the complete system

In [ ]:
print("**CORRECTED AGENTIC RAG WITH PROPER OUTPUT GUARDRAILS - TESTING**")
print("=" * 80)

# Test cases
test_questions = [
    "What is the derivative of x^2 + 3x + 2?",
    "Solve the equation 2x + 5 = 15",
    "Find the integral of sin(x) dx",
    "What is the area of a circle with radius 5?",
    "Explain the Pythagorean theorem with an example"
]

for i, question in enumerate(test_questions, 1):
    print(f"\n**TEST {i}:**")
    result = math_rag_system.solve_math_problem(question)
    print("\n" + "*" * 80)

## Optimization and feedback analysis

In [ ]:
print("\n**FEEDBACK ANALYSIS:**")
print("=" * 80)

# Display feedback summary
if dspy_optimizer.feedback_data:
    df = pd.DataFrame(dspy_optimizer.feedback_data)
    print(f"Total feedback entries: {len(dspy_optimizer.feedback_data)}")
    print(f"Average rating: {df['rating'].mean():.2f}")
    print("Rating distribution:")
    print(df['rating'].value_counts().sort_index())
else:
    print("No feedback collected yet")
